### Connect to your project

NOTE: When running the first cell you will be prompted to pick your Python environment, for the pre-built container choose:
 * **Python Environment** on the first dropdown
 * **Python 3.10.x** on the second dropdown

In [ ]:
!az login --use-device-code

In [1]:
from azure.ai.generative import AIClient
from azure.identity import DefaultAzureCredential

# connects to project defined in the config.json file at the root of the repo
# use "ai init" to update this to point at your project
client = AIClient.from_config(DefaultAzureCredential())

Class AIClient: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class WorkspaceHubOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Retrieve Azure OpenAI and Cognitive Services Connections and Set in Environment   

In [2]:
# Log into the Azure CLI (run az login --use-device code) before running this step!
default_aoai_connection = client.get_default_aoai_connection()
default_aoai_connection.set_current_environment()

# change this if you use different connection name
default_acs_connection = client.connections.get("Default_CognitiveSearch")
default_acs_connection.set_current_environment()

# change these if you use different deployment names
# if you do that, also update the deployment name in qna_simple/langchain_model.py 
chat_model_deployment = "gpt-35-turbo-16k-0613"
embedding_model_deployment = "text-ada-embedding-002-2"


Class WorkspaceConnection: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Build MLIndex Locally

In [ ]:
from azure.ai.generative.operations._index_data_source import LocalSource, ACSOutputConfig
from azure.ai.generative.functions.build_mlindex import build_mlindex

# build the index using the product catalog docs from data/3-product-info
index = build_mlindex(
    output_index_name="product-info-cog-search-index",
    vector_store="azure_cognitive_search",
    embeddings_model = f"azure_open_ai://deployment/{embedding_model_deployment}/model/text-embedding-ada-002",
    data_source_url="https://product_info.com",
    index_input_config=LocalSource(input_data="../../data/3-product-info"),
    acs_config=ACSOutputConfig(
        acs_index_name="product-info-index-test1",
    ),
)

# register the index so that it shows up in the project
client.mlindexes.create_or_update(index)


### Implement co-pilot logic using Semantic Kernel and the MLIndex

In [3]:
from semantic_kernel.connectors.memory.azure_cognitive_search import AzureCognitiveSearchMemoryStore
from semantic_kernel.connectors.memory.azure_cognitive_search import utils as acs_utils

# Create a connection to Azure Cognitive Search using the default ACS connection
memory_store = AzureCognitiveSearchMemoryStore(
    vector_size=1536,
    search_endpoint=default_acs_connection.target,
    admin_key=default_acs_connection.credentials.key,
    custom_schema={
        acs_utils.SEARCH_FIELD_ID_KEY: "id",
        acs_utils.SEARCH_FIELD_TEXT_KEY: "content",
        acs_utils.SEARCH_FIELD_EMBEDDING_KEY: "content_vector_open_ai",
        acs_utils.SEARCH_FIELD_SRC_KEY: "sourcepage",
        acs_utils.SEARCH_FIELD_DESC_KEY: "content",
        acs_utils.SEARCH_FIELD_METADATA_KEY: "meta_json_string"
    }
)

In [4]:
# create a QnA function that retrieves data and uses it as context to the LLM
def qna(question, temperature=0.0, number_of_docs=10):
    import semantic_kernel as sk
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
    from semantic_kernel.planning import StepwisePlanner
    from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import StepwisePlannerConfig
    from plugins.customer_support_plugin.customer_support import CustomerSupport

    import os

    # Hack to support async functions
    import asyncio
    import nest_asyncio
    nest_asyncio.apply()

    # Initialize the kernel
    kernel = sk.Kernel()
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(
            chat_model_deployment,
            os.getenv("OPENAI_API_BASE"),
            os.getenv("OPENAI_API_KEY"),
        )
    )
    
    # Add the customer support plugin to the kernel
    kernel.import_skill(CustomerSupport(
        memory_store = memory_store,
        number_of_docs = number_of_docs,
        embedding_model_deployment = embedding_model_deployment,
        chat_model_deployment=chat_model_deployment,
        temperature=temperature
    ), skill_name="CustomerSupport")

    # Create and run plan based on the customer ask
    planner = StepwisePlanner(kernel, config=StepwisePlannerConfig(max_iterations=5))
    plan = planner.create_plan(question)
    result = asyncio.run(kernel.run_async(plan))

    return {
        "question": question,
        "answer": result.result
    }
    

In [5]:
result = qna("What's the waterproof rating for the tent I just ordered?")
print(result["answer"])

The waterproof rating for the tent is 2000mm.


### Evaluate LLM Response on Larger Dataset

In [85]:
import json
import os

# Loading data
def load_jsonl(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines()]

path = os.path.join(os.getcwd() + "/data.jsonl")
data = load_jsonl(path)

In [86]:
from azure.ai.generative.evaluate import evaluate
from pprint import pprint

# Evaluate the default vs the improved system prompt to see if the improved prompt
# performs consistently better across a larger set of inputs
result = evaluate(
    evaluation_name="baseline-evaluation",
    asset=qna,# model_uri:
    data=data,
    task_type="qa",
    prediction_data="answer",
    truth_data="truth", # Optional
    metrics_config={
        "openai_params": {
            "api_version": "2023-05-15",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_type": "azure",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "deployment_id": chat_model_deployment
        },
        "questions": "question",
        "contexts": "context",
    },
    tracking_uri=client.tracking_uri,
)
pprint(result)

# Print a link to open the evalautions page in AI Studio
print(f"Open in AI Studio: https://ml.azure.com/projectEvaluation?flight=AiStudio,DeployChatWebapp,SkipAADRegistration/projectEvaluation&wsid=/subscriptions/{client.subscription_id}/resourceGroups/{client.resource_group_name}/providers/Microsoft.MachineLearningServices/workspaces/{client.project_name}")

test data is not a file but loaded data
Error logging data as dataset, continuing without it
{'artifacts': {'gpt_coherence': ['3',
                                 '3',
                                 '5',
                                 '5',
                                 '1',
                                 '5',
                                 '3',
                                 '5',
                                 '5',
                                 '3',
                                 '5',
                                 '5',
                                 '1'],
               'gpt_fluency': ['3',
                               '3',
                               '5',
                               '5',
                               '5',
                               '5',
                               '5',
                               '5',
                               '5',
                               '5',
                               '5',
                

### Evaluate copilot performance across different parameters

In [76]:
# Sweep over different values of temperature and number_of_docs to find the best value
# Evaluation results will be logged to project by setting tracking_uri
result = evaluate( 
    evaluation_name="qna-params-eval",
    asset=qna,
    data=data,
    task_type="qa",
    prediction_data="answer",
    truth_data="truth", # Optional
    metrics_config={
        "openai_params": {
            "api_version": "2023-05-15",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_type": "azure",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "deployment_id": chat_model_deployment
        },
        "questions": "question",
        "contexts": "context",
    },
    tracking_uri=client.tracking_uri,
    params={
        "temperature": [0.0, 0,1],
        "number_of_docs": [5, 10]
    }
)

pprint(result)

# Print a link to open the evalautions page in AI Studio
print(f"Open in AI Studio: https://ml.azure.com/projectEvaluation?flight=AiStudio,DeployChatWebapp,SkipAADRegistration/projectEvaluation&wsid=/subscriptions/{client.subscription_id}/resourceGroups/{client.resource_group_name}/providers/Microsoft.MachineLearningServices/workspaces/{client.project_name}")

test data is not a file but loaded data
Error logging data as dataset, continuing without it
test data is not a file but loaded data
Error logging data as dataset, continuing without it
test data is not a file but loaded data
Error logging data as dataset, continuing without it
test data is not a file but loaded data
Error logging data as dataset, continuing without it


Retrying azureml.metrics.text.qa._similarity_utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


test data is not a file but loaded data
Error logging data as dataset, continuing without it
test data is not a file but loaded data
Error logging data as dataset, continuing without it
[{'artifacts': {'gpt_coherence': ['5',
                                  '3',
                                  '5',
                                  '5',
                                  '1',
                                  '5',
                                  '3',
                                  '5',
                                  '5',
                                  '3',
                                  '5',
                                  '5',
                                  '1'],
                'gpt_fluency': ['5',
                                '3',
                                '5',
                                '5',
                                '5',
                                '5',
                                '3',
                                '5',
          

### Deploy Semantic Kernel QA Function to MIR

In [19]:
# download MLIndex files so they can be packaged with deployment code
client.mlindexes.download(name="product-info-cog-search-index", download_path="./qna_simple/mlindex", label="latest")

# set the deployment name to be used, the url needs to be globally unique so include project name
deployment_name = f"{client.project_name}-sk-copilot" 

In [20]:
from azure.ai.generative.entities.deployment import Deployment
from azure.ai.generative.entities.models import LocalModel

deployment = Deployment(
    name=deployment_name,
    model=LocalModel(
        path="./qna_simple",
        conda_file="conda.yaml",
        loader_module="model_loader.py",
    ),
)

deployment = client.deployments.create_or_update(deployment)

System-assigned identity already has access to project. Skipping granting it access to project


Check: endpoint dantaylo-bugbashcli-sk-copilot exists
Uploading mlflow_model (0.02 MBs): 100%|██████████| 16643/16643 [00:00<00:00, 26566.58it/s]




..................................................................................................................................

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


### Invoke Deployment

In [21]:
response = client.deployments.invoke(deployment_name, "./request_file_qna_simple.json")
print(response)

[{"question": "Which tent has the highest rainfly waterproof rating?", "answer": "Sorry, I can only answer questions related to outdoor/camping gear and clothing. How can I assist you with that?\""}]
